## CAS Deep Learning - Computer Vision mit Deep Learning (Part 1)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Convolutional Neural Networks

## Lernziele

- Convolutions: Anwenden auf Bilder
- CNNs: Definieren, Optimieren, Inspizieren

## Setup

Im Folgenden installieren und laden wir die benötigten Python packages. Danach setzten wir die Pfade für den Zugriff auf Daten und spezifizieren einen Output-Folder.

In [ ]:
import os
from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import torch
from tqdm.notebook import tqdm

Mount your google drive to store data and results.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

print(f"In colab: {IN_COLAB}")

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Modifizieren Sie die folgenden Pfade bei Bedarf.

In [ ]:
if IN_COLAB:
    DATA_PATH = Path('/content/drive/MyDrive/bverI/data')
else:
    DATA_PATH = Path('../data')

Install packages not in base Colab environment.

In [ ]:
if IN_COLAB:
    os.system("pip install torchshow torchinfo gdown")
import torchshow as ts

## Convolutions in PyTorch

Als erstes schauen wir an wie wir _convonlutions_ auf Bilder anwenden.

Dazu lesen wir ein Bild ein.

In [ ]:
import io
import requests

url = "https://github.com/pytorch/vision/blob/main/gallery/assets/dog2.jpg?raw=true"
r = requests.get(url, allow_redirects=True)
image = Image.open(io.BytesIO(r.content))

image

Wir können nun _convolutions_ durchführen. Dazu gibt es zwei Möglichkeiten:

- _functional_ Ansatz, mit Funktionen, die _stateless_ sind [nn.functional](https://pytorch.org/docs/stable/nn.functional.html)
- mit Modulen (Objekten), welche einen _state_ haben und beim Einsatz in neuronalen Netzwerken verwendet werden [torch.nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

In [ ]:
import torch
from torch.nn import functional as F
from torchvision.transforms import functional as TF

# convert PIL.Image to torch.tensor (takes care of channel format -> CHW)
input = TF.pil_to_tensor(image).float() / 255.0

# define filter by hand
filter = filter = torch.tensor(
    [   [[1, 0, -1], [1, 0, -1], [1, 0, -1]], # R
        [[1, 0, -1], [1, 0, -1], [1, 0, -1]], # G
        [[1, 0, -1], [1, 0, -1], [1, 0, -1]], # B
    ]).unsqueeze(0).float()

ts.show(filter, show_axis=False)

In [ ]:
# functional approach
result = F.conv2d(input, filter, stride=1, padding=0, dilation=1, groups=1)

# rescale result to visualize it as an image
result_scaled = (result - result.min()) / (result.max() - result.min())

ts.show(result_scaled)

Nun führen wir eine _convolution_ mit einem _module_ aus. [torch.nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

In [ ]:
# module
conv = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=1, groups=1)

result = conv(input)

# rescale result to visualize it as an image
result_scaled = (result - result.min()) / (result.max() - result.min())


ts.show(result_scaled)

**Frage**: Was ist der Unterschied zwischen _functional_ und _module_ Ansatz? Was passiert beim _module_ Ansatz?

YOUR ANSWER HERE

Wenden Sie nun folgende Operationen an auf dem Bild. Alles mit dem _functional_ Ansatz.

- Convolution
- Max Pooling
- Convolution

Sie können den Filter von oben verwenden. Falls das geht.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## CNN auf MNIST (graustufen Bilder)

Im Folgenden werden wir ein CNN auf graustufen Bildern trainieren und einige Eigenschaften von CNNs überprüfen.

### Datensatz

Wir erstellen nun eine modifizierte Variante vom MNIST Datensatz.

In [ ]:
from functools import partial

import torch
import torchvision
from torchvision import transforms

from torch.nn import functional as F

# Define a transform to pad the images to the right and bottom quadrants with zeros (black pixels)
def mnist_transform(image, position="top_left"):
    # Convert the image to a tensor
    image_tensor = transforms.ToTensor()(image)

    if position == "top_left":
        output_tensor= F.pad(image_tensor, (0, 28, 0, 28), "constant", 0)
    elif position == "bottom_right":
        output_tensor= F.pad(image_tensor, (28, 0, 28, 0), "constant", 0)
    return output_tensor

# Create the MNIST dataset with the custom transform
mnist_dataset_train = torchvision.datasets.MNIST(
    root=DATA_PATH,
    train=True,
    download=True,
    transform=lambda x: mnist_transform(x, position="top_left")
)

# Create the MNIST dataset with the custom transform
mnist_dataset_test_tl = torchvision.datasets.MNIST(
    root=DATA_PATH,
    train=False,
    download=True,
    transform=lambda x: mnist_transform(x, position="top_left")
)

# Create the MNIST dataset with the custom transform
mnist_dataset_test_br = torchvision.datasets.MNIST(
    root=DATA_PATH,
    train=False,
    download=True,
    transform=lambda x: mnist_transform(x, position="bottom_right")
)

Nun schauen wir uns einige Datenpunkte an.

In [ ]:
mnist_train = torch.utils.data.DataLoader(
    mnist_dataset_train, batch_size=12, shuffle=True, num_workers=4)

# Let's check the first batch
images, labels = next(iter(mnist_train))
import torchshow
ts.show(images)

**Frage**: Was fällt auf?

YOUR ANSWER HERE

### Definition CNN

Definieren Sie ein CNN mit folgender Architektur:

- Input Shape: (1, 28 *  2, 28 *2)
- Convolution: 8 Filters, Kernel-Size 5x5
- Max Pooling: Stride 2, Kernel-Size 2
- Convolution: 16 Filter, Kernel-Size 5x5
- Max Pooling: Stride 2, Kernel-Size 2
- FC: 32 Neuronen
- FC: 16 Neuronen
- FC: 10 Neuronen (für 10 Klassen)

Benutzen Sie ReLU Aktivierungen nach jedem Layer.

Definieren Sie eine Klasse, die von `torch.nn.Module` erbt und instanzieren Sie Ihr Netzwerk.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchinfo


class Net(nn.Module):
    def __init__(self, num_channel=8):
        super().__init__()
        # self.conv1 = nn.Conv2d(...)
        # self.pool = nn.Maxnn.MaxPool2d(...)
        # ...
        # YOUR CODE HERE
        raise NotImplementedError()

    def forward(self, x):
        # x = self....
        # x = self....
        # YOUR CODE HERE
        raise NotImplementedError()

    

net = Net()

print(net)
print(torchinfo.summary(net, input_size=(1, 1, 56, 56)))


### Trainieren eines CNN

Wir werden später noch genauer sehen, warum wir die folgenden Objekte benötigen. Diese definieren die _Loss_ Funktion und den Optimisierungsalgorithmus.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Wir erstellen nun einen Trainingsloop für unser Modell und trainieren dieses gleich.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Wir evaluieren unser Modell auf den Testdaten.


In [ ]:
testloader_tl = torch.utils.data.DataLoader(
    mnist_dataset_test_tl, batch_size=32, shuffle=False)

images, labels = next(iter(testloader_tl))

ts.show(images)

In [ ]:
with torch.no_grad():
    predictions = F.softmax(net.forward(images), -1).argmax(-1)

accuracy = (predictions == labels).to(float).mean()

print(f"Accuracy: {accuracy:.2f}")

Nun verwenden wir die folgenden Testdaten.

In [ ]:
testloader_br = torch.utils.data.DataLoader(
    mnist_dataset_test_br, batch_size=32, shuffle=False)

images, labels = next(iter(testloader_br))

ts.show(images)

**Frage**: Was schätzen Sie? Wie gut ist das Modell in diesem Fall?

YOUR ANSWER HERE

Überprüfen Sie Ihre Antwort:

In [ ]:
with torch.no_grad():
    predictions = F.softmax(net.forward(images), -1).argmax(-1)

accuracy = (predictions == labels).to(float).mean()

print(f"Accuracy: {accuracy:.2f}")

**Frage**: Sind Sie überrascht? Was ist passiert?

YOUR ANSWER HERE